In [99]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chi2

# Átomo de Hidrogeno y Deuterio.

In [100]:
def Minimos_cuadrados(x,y,n=1):
    l = x.shape[0]
    b = y
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    AT = np.dot(A.T,A)
    bT = np.dot(A.T,b)

    xsol = np.linalg.solve(AT,bT)
    
    return xsol

def GetModel(x,p):
    
    y = 0.
    for i in range(len(p)):
        y += p[i]*x**i
        
    return y

def GetError(x,y,p,n=1):
    
    l = x.shape[0]
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    # Residuos
    R = y - np.dot(A,p)
    
    sigma2 = np.dot(R.T,R)/(len(y)-len(p))
    
    Cov = sigma2*np.linalg.inv(np.dot(A.T,A))
    
    R2 = None
    if np.shape(Cov) == (2,2):
        R2 = (Cov[0][1]**2)/(Cov[0][0]*Cov[1][1])

    return (Cov,R2,R)

def Get_sigma(Cov):
    sigmaai = np.sqrt(np.diagonal(Cov))
    return sigmaai[::-1]

def Error_porcentual(X_real,X_exp):
    return (np.abs(X_exp - X_real)/X_real)*100

def presicion(d,v):
    p = (d/v)
    if p <= 0.1:
        return p,True
    else:
        return p,False
    
def exactitud(xr,xm,d):
    e = (np.abs(xr-xm))
    if e <= (2.5*d):
        return e,True
    else:
        return e,False

X = sym.Symbol('x',real=True)
Y = sym.Symbol('y',real=True)

In [101]:
c = 299798458*1e09 #nm/s
me = 0.51099895*1e06/(c**2) #eV/c**2
mp = 938.27208816*1e06/(c**2) #eV/c**2
mn = 939.56542052*1e06/(c**2) #eV/c**2
h = 4.136667696*1e-15 #eV*s
h_barra = 6.582119569*1e-16 #eV*s
e_0 = 55.26349406*1e-03 #e**2/(cV*nm)
e = 1 #e
a_0 = 0.0529177210903 #nm
Ry = 13.605693122994 #eV

def Mr(m1=me,m2=mp):
    return (m1*m2)/(m1+m2)

def coef_Mr(M,m):
    return M/(m+M)

def LongOnda(Z=1,M=mp,m=me,ni=3,nf=2):
    return ((h*c)/(Ry*(Z**2)*coef_Mr(M,m)))*(((1/(nf**2)) - (1/(ni**2)))**(-1))

H_a = LongOnda()
D_a = LongOnda(1,mp+mn,me,3,2)
d = np.abs(H_a-D_a)
d

0.17883481948592816

# Datos.

In [102]:
d_Ne_H = 1028 #píxeles
d_H_D = 61 #píxeles
Ne_ = 653.28824 #nm
H_a = 656.2795 #nm
D_a = 656.1012 #nm
d_px = 0.5
d = np.abs(H_a-D_a)
d

0.17830000000003565

In [103]:
Error_porcentual(Mr(mp,me),me)

0.05446170214890631

In [104]:
Error_porcentual(mp+mn,2*mp)

0.06887349698904745

es mejor tomar Mr(mp,me) igual me

# Análisis

calibración

In [105]:
# Para H y 2H
def px_to_nm(px):
    return px*(np.abs(Ne_-H_a)/d_Ne_H)

d_ = px_to_nm(d_H_D)
d_

0.1774969455252925

In [106]:
# Desviación estandard de d_
SD_nm = np.abs(Ne_-H_a)*(d_px)*np.sqrt(1+(1/((d_Ne_H*d_H_D)**2)))
SD_nm

1.4956300001901783

In [107]:
presicion(SD_nm,d_)

(8.426229509268136, False)

In [108]:
exactitud(d,d_,SD_nm)

(0.0008030544747431589, True)

constante de proporcionalidad entre protón y electrón: mp = a*me, mp=mn

In [109]:
m_p = ((36*h*c)/(10*Ry*d_))*me
m_p

1.0510713385452674e-26

In [110]:
#mp_ = (2*m_p)-mn
#mp_

1.0567770032762573e-26

In [111]:
mp

1.0439267050906e-26